<a href="https://colab.research.google.com/github/ShadaabHasan/MP305/blob/main/Lab2_Network_Flows_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MP305 Network Flow Models II

In [ ]:
from IPython.display import display, Math, Latex

## Overview

This file contains a number of Python functions for finding the maximal flow through a network $G$ subject to minimal cost using the Ford Fulkerson Algorithm.

The network graph $G$ is stored in a set `G` of two element tuples `(i,j)` describing the directed arcs $(i,j)$ of $G$.

It is assumed that node number $1$ is the source and the greatest node `Nsink` is the sink.  
Thus `G={(1,2),(2,3),(1,3)}`  describes a network with 3 nodes where node 1 is the source and node 3 is the sink.

The capacity $c(i,j)$, flow $\phi(i,j)$ and cost $l(i,j)$ of the arc $(i,j)$ of $G$ are stored in `c[i][j]`, `phi[i][j]` and `l[i][j]`.
Here `c`, `phi` and `l` are Python lists.

## Python Functions
### The `Initialise(G)` function
Having defined the network $G$, initialise `c`, `phi` and `l` values to zero via the `Initialise` function before defining their values in any particular example.  The global variable `Nsink`,  the sink node of $G$, is also found by the `Initialise` function .

### The main `Iterate(G)`function
This implements the full algorithm to find the maximum flow with minimal cost.

## The `Iterate(G)` function is based on a number of other Python functions:

### `Flows(G)`
This checks for conservation of flow and prints out all of the current flows for G and the total cost of this flow.

###  `Links(G)`
This finds all arcs `(i,j1)`, ` (i,j2)`,  ... *out* of node `i` of `G`.  The nodes `j1,j2,..` are stored in a global list of sets `Out`.

###  `SourceSink(G)`
This finds all of the paths from source to sink in any network `G` and results in a global set `SinkPaths` of such paths.

###  `IncremNet(G)`
This finds the Incremental Network `Gp` associated with the current flow of the network `G`.

###  `Newflows(G)`
This updates the flows `phi` of `G` according to the best chain found through `Gp`. If the maximal flow is reached, then this is indicated and the maximum flow value is outputed. Otherwise, the output is: the change in flow (`eps`), the cost of the best chain, and the best chain.

###  `Iterate(G)`
Implements the full algorithm to find the maximum flow with minimal cost.
The output is as follows:

(1) The incremental network `Gp`.

(2) The paths through `Gp` from source to sink.

(3) The output of `Newflows(G)`.

(4) The output of `Flows(G)` giving the current flows and cost of `G`.


In [ ]:
def Initialise (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of c,phi and l initialised to zero")


In [ ]:
def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in Gin:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)


In [ ]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [ ]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [ ]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost:
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", besteps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [ ]:
def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

In [ ]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

# Q 1. Power Station/Coal Field Supply
### Find the maximal flow for minimal cost for the network below where (capacity,cost) is shown:

![Network](https://github.com/mcgettrick/mp305/blob/main/Lab2_1.jpg?raw=1)


## This is the Coalfield/Power station supply problem discussed in the notes
### * Find the incremental network, its capacities and costs at each iteration.

In [ ]:
#define the nodes
G={(1,2),(1,3),(2,4),(2,5),(3,4),(3,5),(4,6),(5,6)}

In [ ]:
#initialise the network and set all values to 0
Initialise(G)

All values of c,phi and l initialised to zero


## Input capacities

In [ ]:
#define capacities
c[1][2]=3; c[1][3]=5; c[2][4]=3; c[2][5]=3
c[3][5]=5; c[3][4]=5; c[4][6]=2; c[5][6]=5

## Input costs

In [ ]:
#define the costs
l[1][2]=5; l[1][3]=3;
l[2][4]=3
l[2][5]=6; l[3][4]=5
l[3][5]=9

In [ ]:
#initialise the flow to 0
Flows(G)

(2, 4)  has flow  0
(1, 2)  has flow  0
(3, 4)  has flow  0
(4, 6)  has flow  0
(5, 6)  has flow  0
(2, 5)  has flow  0
(1, 3)  has flow  0
(3, 5)  has flow  0
Total Cost is  0


In [ ]:
#first iteration
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (2, 5), (3, 4), (5, 6), (4, 6), (1, 3), (3, 5)}
Paths from source to sink:  {(1, 2, 5, 6), (1, 3, 4, 6), (1, 2, 4, 6), (1, 3, 5, 6)}
A best path in Gp is  (1, 3, 4, 6)  of minimum cost  8
The min capacity on this path is epsilon  2
The min cost is  8
(2, 4)  flow =  0
(1, 2)  flow =  0
(3, 4)  flow =  2
(4, 6)  flow =  2
(5, 6)  flow =  0
(2, 5)  flow =  0
(1, 3)  flow =  2
(3, 5)  flow =  0


### Continue to iterate until max flow for minimal cost found

In [ ]:
#second iteration
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (3, 4), (4, 3), (3, 1), (6, 4), (5, 6), (2, 5), (1, 3), (3, 5)}
Paths from source to sink:  {(1, 2, 5, 6), (1, 2, 4, 3, 5, 6), (1, 3, 5, 6)}
A best path in Gp is  (1, 2, 5, 6)  of minimum cost  11
The min capacity on this path is epsilon  3
The min cost is  11
(2, 4)  flow =  0
(1, 2)  flow =  3
(3, 4)  flow =  2
(4, 6)  flow =  2
(5, 6)  flow =  3
(2, 5)  flow =  3
(1, 3)  flow =  2
(3, 5)  flow =  0


In [ ]:
#third iteration
Iterate(G)

Incremental Network: {(2, 4), (2, 1), (3, 4), (4, 3), (6, 5), (3, 1), (6, 4), (5, 6), (1, 3), (3, 5), (5, 2)}
Paths from source to sink:  {(1, 3, 5, 6)}
A best path in Gp is  (1, 3, 5, 6)  of minimum cost  12
The min capacity on this path is epsilon  2
The min cost is  12
(2, 4)  flow =  0
(1, 2)  flow =  3
(3, 4)  flow =  2
(4, 6)  flow =  2
(5, 6)  flow =  5
(2, 5)  flow =  3
(1, 3)  flow =  4
(3, 5)  flow =  2


In [ ]:
#fourth iteration
Iterate(G)

Incremental Network: {(2, 4), (2, 1), (3, 4), (4, 3), (6, 5), (3, 1), (6, 4), (5, 3), (1, 3), (3, 5), (5, 2)}
Paths from source to sink:  set()
Maximal flow found: 7  with minimal cost  73
(2, 4)  flow =  0
(1, 2)  flow =  3
(3, 4)  flow =  2
(4, 6)  flow =  2
(5, 6)  flow =  5
(2, 5)  flow =  3
(1, 3)  flow =  4
(3, 5)  flow =  2


# Q2. (*)  
## A road network is shown below with the capacity and time taken per car on each road indicated.
![Network](https://github.com/mcgettrick/mp305/blob/main/Lab2_2.jpg?raw=1)


## * Find the maximal flow through the network for minimal total travel time for all cars from A to B.
## * Compare this to the flow from B to A.

### Note: You can use the network and capacitites you created in the first lab Network Flows I, Question 3.

# A TO B

In [ ]:
# Define the network from A to B
G2={(1,2),(2,1),(1,3),(3,1),(2,6),(3,4),(3,7),(4,3),(4,2),(4,5),(5,4),(6,5),(6,8),(7,5),(7,8),(8,6),(8,7)}

In [ ]:
#initialise the network and set all the values to 0
Initialise(G2)

All values of c,phi and l initialised to zero


In [ ]:
#Define the capacities as shown in the network
c[1][2]=5; c[2][1]=5;
c[1][3]=4; c[3][1]=4;
c[2][6]=3;
c[3][4]=5; c[4][3]=5;
c[3][7]=3;
c[4][2]=4;
c[4][5]=5; c[5][4]=5;
c[6][5]=2;
c[6][8]=6; c[8][6]=6;
c[7][5]=4;
c[7][8]=4; c[8][7]=4;

In [ ]:
#Define the costs as shown in the network
l[1][2]=3; l[2][1]=3;
l[1][3]=2; l[3][1]=2;
l[2][6]=2;
l[3][4]=1; l[4][3]=1;
l[3][7]=2;
l[4][2]=3;
l[4][5]=2; l[5][4]=2;
l[6][5]=3;
l[6][8]=3; l[8][6]=3;
l[7][5]=3;
l[7][8]=1; l[8][7]=1;

In [ ]:
#Initialise the flow as 0 through the network
Flows(G2)

(1, 2)  has flow  0
(2, 1)  has flow  0
(3, 4)  has flow  0
(4, 3)  has flow  0
(3, 1)  has flow  0
(3, 7)  has flow  0
(5, 4)  has flow  0
(8, 6)  has flow  0
(1, 3)  has flow  0
(6, 5)  has flow  0
(8, 7)  has flow  0
(6, 8)  has flow  0
(4, 2)  has flow  0
(4, 5)  has flow  0
(2, 6)  has flow  0
(7, 5)  has flow  0
(7, 8)  has flow  0
Total Cost is  0


In [ ]:
# First iteration
Iterate(G2)

Incremental Network: {(1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (3, 7), (5, 4), (8, 6), (1, 3), (6, 5), (8, 7), (6, 8), (4, 2), (4, 5), (2, 6), (7, 5), (7, 8)}
Paths from source to sink:  {(1, 3, 7, 8), (1, 3, 4, 2, 6, 8), (1, 2, 6, 5, 4, 3, 7, 8), (1, 2, 6, 8), (1, 3, 7, 5, 4, 2, 6, 8)}
A best path in Gp is  (1, 3, 7, 8)  of minimum cost  5
The min capacity on this path is epsilon  3
The min cost is  5
(1, 2)  flow =  0
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(5, 4)  flow =  0
(8, 6)  flow =  0
(1, 3)  flow =  3
(6, 5)  flow =  0
(8, 7)  flow =  0
(6, 8)  flow =  0
(4, 2)  flow =  0
(4, 5)  flow =  0
(2, 6)  flow =  0
(7, 5)  flow =  0
(7, 8)  flow =  3


In [ ]:
# Second iteration
Iterate(G2)

Incremental Network: {(1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (5, 4), (7, 3), (8, 6), (1, 3), (6, 5), (8, 7), (6, 8), (4, 2), (4, 5), (2, 6), (7, 5), (7, 8)}
Paths from source to sink:  {(1, 2, 6, 8), (1, 3, 4, 2, 6, 8)}
A best path in Gp is  (1, 2, 6, 8)  of minimum cost  8
The min capacity on this path is epsilon  3
The min cost is  8
(1, 2)  flow =  3
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(5, 4)  flow =  0
(8, 6)  flow =  0
(1, 3)  flow =  3
(6, 5)  flow =  0
(8, 7)  flow =  0
(6, 8)  flow =  3
(4, 2)  flow =  0
(4, 5)  flow =  0
(2, 6)  flow =  3
(7, 5)  flow =  0
(7, 8)  flow =  3


In [ ]:
# Third iteration
Iterate(G2)

Incremental Network: {(1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (5, 4), (7, 3), (8, 6), (1, 3), (6, 2), (6, 5), (8, 7), (6, 8), (4, 2), (4, 5), (7, 5), (7, 8)}
Paths from source to sink:  set()
Maximal flow found: 6  with minimal cost  39
(1, 2)  flow =  3
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(5, 4)  flow =  0
(8, 6)  flow =  0
(1, 3)  flow =  3
(6, 5)  flow =  0
(8, 7)  flow =  0
(6, 8)  flow =  3
(4, 2)  flow =  0
(4, 5)  flow =  0
(2, 6)  flow =  3
(7, 5)  flow =  0
(7, 8)  flow =  3


# B to A

In [ ]:
# Define the network from B to A
G2_reversed={(1,2),(2,1),(1,3),(3,1),(2,4),(3,4),(4,5),(5,4),(5,6),(5,7),(7,5),(6,2),(7,3),(6,8),(7,8),(8,7),(8,6)}

In [ ]:
#initialise the network and set all the values to 0
Initialise(G2_reversed)

All values of c,phi and l initialised to zero


In [ ]:
# Define the capacites as shown in the network
c[1][2]=6; c[2][1]=6;
c[1][3]=4; c[3][1]=4;
c[2][4]=2;
c[3][4]=4;
c[4][5]=5;
c[5][4]=5;
c[5][6]=4;
c[5][7]=5; c[7][5]=5;
c[6][2]=3;
c[7][3]=3;
c[6][8]=5; c[8][6]=5;
c[7][8]=4; c[8][7]=4;


In [ ]:
# Define the cost as shown in the network
l[1][2]=3; l[2][1]=3;
l[1][3]=1; l[3][1]=1;
l[2][4]=3;
l[3][4]=3;
l[4][5]=2; l[5][4]=2;
l[5][6]=3;
l[5][7]=1; l[7][5]=1;
l[6][2]=2;
l[7][3]=2;
l[6][8]=3; l[8][6]=3;
l[7][8]=2; l[8][7]=2;


In [ ]:
#Initialise the flow as 0 through the network
Flows(G2_reversed)

(2, 4)  has flow  0
(1, 2)  has flow  0
(2, 1)  has flow  0
(3, 4)  has flow  0
(3, 1)  has flow  0
(5, 4)  has flow  0
(5, 7)  has flow  0
(7, 3)  has flow  0
(8, 6)  has flow  0
(1, 3)  has flow  0
(6, 2)  has flow  0
(8, 7)  has flow  0
(6, 8)  has flow  0
(4, 5)  has flow  0
(5, 6)  has flow  0
(7, 5)  has flow  0
(7, 8)  has flow  0
Total Cost is  0


In [ ]:
#First iteration
Iterate(G2_reversed)

Incremental Network: {(2, 4), (1, 2), (2, 1), (3, 4), (3, 1), (5, 4), (5, 7), (7, 3), (8, 6), (1, 3), (6, 2), (8, 7), (6, 8), (4, 5), (5, 6), (7, 5), (7, 8)}
Paths from source to sink:  {(1, 2, 4, 5, 7, 8), (1, 3, 4, 5, 6, 8), (1, 2, 4, 5, 6, 8), (1, 3, 4, 5, 7, 8)}
A best path in Gp is  (1, 3, 4, 5, 7, 8)  of minimum cost  9
The min capacity on this path is epsilon  4
The min cost is  9
(2, 4)  flow =  0
(1, 2)  flow =  0
(2, 1)  flow =  0
(3, 4)  flow =  4
(3, 1)  flow =  0
(5, 4)  flow =  0
(5, 7)  flow =  4
(7, 3)  flow =  0
(8, 6)  flow =  0
(1, 3)  flow =  4
(6, 2)  flow =  0
(8, 7)  flow =  0
(6, 8)  flow =  0
(4, 5)  flow =  4
(5, 6)  flow =  0
(7, 5)  flow =  0
(7, 8)  flow =  4


In [ ]:
#Second Iteration
Iterate(G2_reversed)

Incremental Network: {(2, 4), (1, 2), (6, 2), (2, 1), (4, 3), (3, 1), (8, 7), (6, 8), (5, 4), (5, 7), (7, 3), (4, 5), (5, 6), (8, 6), (7, 5)}
Paths from source to sink:  {(1, 2, 4, 5, 6, 8)}
A best path in Gp is  (1, 2, 4, 5, 6, 8)  of minimum cost  14
The min capacity on this path is epsilon  1
The min cost is  14
(2, 4)  flow =  1
(1, 2)  flow =  1
(2, 1)  flow =  0
(3, 4)  flow =  4
(3, 1)  flow =  0
(5, 4)  flow =  0
(5, 7)  flow =  4
(7, 3)  flow =  0
(8, 6)  flow =  0
(1, 3)  flow =  4
(6, 2)  flow =  0
(8, 7)  flow =  0
(6, 8)  flow =  1
(4, 5)  flow =  5
(5, 6)  flow =  1
(7, 5)  flow =  0
(7, 8)  flow =  4


In [ ]:
# Third iteration
Iterate(G2_reversed)

Incremental Network: {(2, 4), (1, 2), (6, 2), (2, 1), (6, 5), (4, 3), (3, 1), (8, 7), (6, 8), (5, 4), (5, 7), (4, 2), (7, 3), (5, 6), (8, 6), (7, 5)}
Paths from source to sink:  set()
Maximal flow found: 5  with minimal cost  50
(2, 4)  flow =  1
(1, 2)  flow =  1
(2, 1)  flow =  0
(3, 4)  flow =  4
(3, 1)  flow =  0
(5, 4)  flow =  0
(5, 7)  flow =  4
(7, 3)  flow =  0
(8, 6)  flow =  0
(1, 3)  flow =  4
(6, 2)  flow =  0
(8, 7)  flow =  0
(6, 8)  flow =  1
(4, 5)  flow =  5
(5, 6)  flow =  1
(7, 5)  flow =  0
(7, 8)  flow =  4


## Comparing the flow from A to B and the reverse flow B to A, we can see that the maximal flow from A to B is 6 which is more as compared to B to A and the minimal flow from A to B is 39 which is less as compared to B to A

 # Q.3 (*) Soft Drinks Stock Control
 ## A soft drinks firm buys fruit at the beginning of each month $i$ at a cost per 100kg $p_i$ in units of 1000 Euro.
 ## The firm can store up 2000kg of fruit at any given time where cost in units 1000 Euro of refrigeration per month per 100kg is $r_i$.
## The consumption requirements are $c_i$ per month in 100kg units.

## Based on last year's figures,  the following estimates have been made:
\begin{array}{|l|l|l|l|l|l|l|l|l|l|l|l|l|}
\hline
i & Jan & Feb & Mar & April & May & June & July & Aug & Sept & Oct & Nov &
Dec \\ \hline
p_{i} & 18 & 17 & 17 & 15 & 12 & 8 & 7 & 6 & 9 & 12 & 14 & 17 \\ \hline
r_{i} & 1 & 1 & 2 & 2 & 3 & 5 & 6 & 6 & 5 & 3 & 2 & 1 \\ \hline
c_{i} & 9 & 6 & 6 & 7 & 11 & 14 & 16 & 18 & 15 & 10 & 7 & 6 \\ \hline
\end{array}

## (a) Find the best purchasing schedule starting from January based on these estimates assuming that the firm has no fruit in storage on Jan 1st.
## How much will fruit cost for the year and how much will be spent on refrigeration?

## (b) Suppose that the firm has 500kg of fruit in storage at the beginning of the year.
## What is the best purchasing schedule that ensures that 500kg of fruit is again in storage at the very end of the year?

#Part A

In [ ]:
#define the network
G3 ={(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(1,11),(1,12),(1,13),(2,14),(3,15),(4,16),(5,17),(6,18),(7,19),(8,20),(9,21),(10,22),(11,23),(12,24),(13,25),(14,3),(15,4),(16,5),(17,6),(18,7),(19,8),(20,9),(21,10),(22,11),(23,12),(24,13),(14,26),(15,26),(16,26),(17,26),(18,26),(19,26),(20,26),(21,26),(22,26),(23,26),(24,26),(25,26)}

In [ ]:
#initialise the network and set the values to 0
Initialise(G3)

All values of c,phi and l initialised to zero


In [ ]:
#defining capacities
#from source to firm
c[1][2]=20
c[1][3]=20
c[1][4]=20
c[1][5]=20
c[1][6]=20
c[1][7]=20
c[1][8]=20
c[1][9]=20
c[1][10]=20
c[1][11]=20
c[1][12]=20
c[1][13]=20
c[2][14]=20
c[3][15]=20
c[4][16]=20
c[5][17]=20
c[6][18]=20
c[7][19]=20
c[8][20]=20
c[9][21]=20
c[10][22]=20
c[11][23]=20
c[12][24]=20
c[13][25]=20

#leftover
c[14][3]=20
c[15][4]=20
c[16][5]=20
c[17][6]=20
c[18][7]=20
c[19][8]=20
c[20][9]=20
c[21][10]=20
c[22][11]=20
c[23][12]=20
c[24][13]=20

#storage to consumption
c[14][26]=9
c[15][26]=6
c[16][26]=6
c[17][26]=7
c[18][26]=11
c[19][26]=14
c[20][26]=16
c[21][26]=18
c[22][26]=15
c[23][26]=10
c[24][26]=7
c[25][26]=6

In [ ]:
#defining cost
#from source to firm
l[1][2]=18
l[1][3]=17
l[1][4]=17
l[1][5]=15
l[1][6]=12
l[1][7]=8
l[1][8]=7
l[1][9]=6
l[1][10]=9
l[1][11]=12
l[1][12]=14
l[1][13]=17
l[2][14]=0
l[3][15]=0
l[4][16]=0
l[5][17]=0
l[6][18]=0
l[7][19]=0
l[8][20]=0
l[9][21]=0
l[10][22]=0
l[11][23]=0
l[12][24]=0
l[13][25]=0

#leftover
l[14][3]=1
l[15][4]=1
l[16][5]=2
l[17][6]=2
l[18][7]=3
l[19][8]=5
l[20][9]=6
l[21][10]=6
l[22][11]=5
l[23][12]=3
l[24][13]=2

#storage to consumption
l[14][26]=0
l[15][26]=0
l[16][26]=0
l[17][26]=0
l[18][26]=0
l[19][26]=0
l[20][26]=0
l[21][26]=0
l[22][26]=0
l[23][26]=0
l[24][26]=0
l[25][26]=0

In [ ]:
#initialise the flow to 0
Flows(G3)

(6, 18)  has flow  0
(21, 10)  has flow  0
(16, 26)  has flow  0
(22, 11)  has flow  0
(18, 26)  has flow  0
(20, 26)  has flow  0
(17, 6)  has flow  0
(22, 26)  has flow  0
(1, 6)  has flow  0
(1, 3)  has flow  0
(1, 9)  has flow  0
(2, 14)  has flow  0
(1, 12)  has flow  0
(11, 23)  has flow  0
(18, 7)  has flow  0
(24, 26)  has flow  0
(7, 19)  has flow  0
(15, 26)  has flow  0
(14, 3)  has flow  0
(23, 12)  has flow  0
(3, 15)  has flow  0
(12, 24)  has flow  0
(19, 8)  has flow  0
(1, 2)  has flow  0
(17, 26)  has flow  0
(1, 5)  has flow  0
(1, 11)  has flow  0
(15, 4)  has flow  0
(8, 20)  has flow  0
(1, 8)  has flow  0
(19, 26)  has flow  0
(24, 13)  has flow  0
(20, 9)  has flow  0
(13, 25)  has flow  0
(4, 16)  has flow  0
(21, 26)  has flow  0
(23, 26)  has flow  0
(5, 17)  has flow  0
(14, 26)  has flow  0
(1, 4)  has flow  0
(9, 21)  has flow  0
(1, 7)  has flow  0
(1, 13)  has flow  0
(16, 5)  has flow  0
(1, 10)  has flow  0
(10, 22)  has flow  0
(25, 26)  has flow  0
T

In [ ]:
#First iteration
Iterate(G3)

Incremental Network: {(6, 18), (21, 10), (16, 26), (22, 11), (18, 26), (20, 26), (17, 6), (22, 26), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (18, 7), (24, 26), (7, 19), (15, 26), (14, 3), (23, 12), (12, 24), (3, 15), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (19, 26), (24, 13), (20, 9), (13, 25), (4, 16), (21, 26), (23, 26), (5, 17), (14, 26), (1, 4), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (25, 26)}
Paths from source to sink:  {(1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 26), (1, 2, 14, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 3, 15, 4, 16, 5, 1

In [ ]:
#Second Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (21, 10), (16, 26), (22, 11), (18, 26), (20, 26), (17, 6), (22, 26), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (18, 7), (24, 26), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (9, 1), (12, 24), (3, 15), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (19, 26), (24, 13), (20, 9), (13, 25), (4, 16), (23, 26), (5, 17), (14, 26), (1, 4), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (25, 26)}
Paths from source to sink:  {(1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 2, 14, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11

In [ ]:
#Third Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (21, 10), (16, 26), (20, 8), (22, 11), (18, 26), (17, 6), (22, 26), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (18, 7), (26, 20), (24, 26), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (9, 1), (12, 24), (3, 15), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (19, 26), (24, 13), (20, 9), (13, 25), (4, 16), (8, 1), (23, 26), (5, 17), (14, 26), (1, 4), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (25, 26)}
Paths from source to sink:  {(1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 26), (1, 6, 18, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24

In [ ]:
#Fourth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (21, 10), (16, 26), (20, 8), (22, 11), (18, 26), (17, 6), (22, 26), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (7, 1), (18, 7), (26, 20), (24, 26), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (9, 1), (12, 24), (3, 15), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (24, 13), (26, 19), (20, 9), (13, 25), (4, 16), (8, 1), (19, 7), (23, 26), (5, 17), (14, 26), (1, 4), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (25, 26)}
Paths from source to sink:  {(1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 26), (1, 6, 18, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 2

In [ ]:
#Fifth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (21, 10), (16, 26), (20, 8), (22, 11), (18, 26), (17, 6), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (7, 1), (18, 7), (26, 20), (24, 26), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (24, 13), (26, 19), (20, 9), (13, 25), (26, 22), (4, 16), (8, 1), (19, 7), (23, 26), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (25, 26)}
Paths from source to sink:  {(1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 6, 18, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 3, 15, 4, 16, 5, 17

In [ ]:
#Sixth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (16, 26), (21, 10), (20, 8), (22, 11), (17, 6), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (7, 1), (18, 7), (26, 20), (24, 26), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (6, 1), (15, 4), (8, 20), (1, 8), (24, 13), (18, 6), (26, 19), (20, 9), (13, 25), (26, 22), (4, 16), (8, 1), (19, 7), (23, 26), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (26, 18), (25, 26)}
Paths from source to sink:  {(1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 2

In [ ]:
#Seventh Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (16, 26), (21, 10), (20, 8), (22, 11), (17, 6), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (7, 1), (18, 7), (26, 20), (24, 26), (26, 23), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (11, 1), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (6, 1), (15, 4), (8, 20), (1, 8), (24, 13), (18, 6), (26, 19), (20, 9), (13, 25), (26, 22), (23, 11), (4, 16), (8, 1), (19, 7), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (10, 22), (26, 18), (25, 26)}
Paths from source to sink:  {(1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 26), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 26), (1, 4, 16, 26), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 

In [ ]:
#Eigth iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (16, 26), (21, 10), (12, 1), (22, 11), (26, 24), (20, 8), (17, 6), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (7, 1), (18, 7), (26, 20), (21, 9), (26, 23), (7, 19), (15, 26), (14, 3), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (11, 1), (19, 8), (1, 2), (17, 26), (1, 5), (1, 11), (6, 1), (15, 4), (8, 20), (1, 8), (24, 13), (18, 6), (26, 19), (20, 9), (13, 25), (26, 22), (23, 11), (4, 16), (8, 1), (19, 7), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (24, 12), (1, 10), (10, 22), (26, 18), (25, 26)}
Paths from source to sink:  {(1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 26), (1, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 3, 15, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 

In [ ]:
#Ninth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (16, 26), (21, 10), (12, 1), (22, 11), (26, 24), (20, 8), (5, 1), (17, 6), (1, 6), (1, 3), (1, 9), (2, 14), (1, 12), (11, 23), (7, 1), (18, 7), (26, 20), (26, 17), (26, 23), (21, 9), (7, 19), (15, 26), (14, 3), (23, 12), (22, 10), (17, 5), (9, 1), (12, 24), (3, 15), (11, 1), (19, 8), (1, 2), (1, 5), (1, 11), (6, 1), (15, 4), (8, 20), (1, 8), (24, 13), (18, 6), (26, 19), (20, 9), (13, 25), (26, 22), (23, 11), (4, 16), (8, 1), (19, 7), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (24, 12), (1, 10), (10, 22), (26, 18), (25, 26)}
Paths from source to sink:  {(1, 2, 14, 3, 15, 4, 16, 26), (1, 12, 24, 13, 25, 26), (1, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 26), (1, 3, 15, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26),

In [ ]:
#Tenth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (12, 1), (22, 11), (5, 1), (1, 9), (11, 23), (7, 1), (21, 9), (26, 23), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (1, 2), (1, 11), (1, 8), (20, 9), (26, 25), (13, 25), (26, 22), (8, 1), (14, 26), (1, 4), (9, 21), (1, 13), (24, 12), (1, 10), (10, 22), (26, 18), (16, 26), (21, 10), (26, 24), (20, 8), (17, 6), (1, 6), (1, 3), (2, 14), (1, 12), (25, 13), (18, 7), (26, 20), (26, 17), (15, 26), (14, 3), (12, 24), (11, 1), (1, 5), (15, 4), (6, 1), (8, 20), (24, 13), (18, 6), (26, 19), (23, 11), (4, 16), (19, 7), (5, 17), (10, 1), (1, 7), (16, 5), (13, 24)}
Paths from source to sink:  {(1, 4, 16, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 26), (1, 3, 15, 26), (1, 3, 15, 4, 16, 26)}
A best path in Gp is  (1, 4, 16, 26)  of minimum cost  17
The min capacity on this path is epsilon  6
The min cost is  17
(6, 18)  flow =  11
(21, 10)  flow =  0
(16, 26)  flow =  6
(22, 11)  flow =  0
(18, 26)  flow =  11
(20, 

In [ ]:
#Eleventh Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (12, 1), (22, 11), (5, 1), (1, 9), (11, 23), (7, 1), (21, 9), (26, 23), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (1, 2), (1, 11), (1, 8), (26, 16), (20, 9), (26, 25), (13, 25), (26, 22), (4, 1), (8, 1), (14, 26), (1, 4), (9, 21), (1, 13), (24, 12), (1, 10), (10, 22), (26, 18), (21, 10), (26, 24), (20, 8), (17, 6), (1, 6), (1, 3), (16, 4), (2, 14), (1, 12), (25, 13), (18, 7), (26, 20), (26, 17), (15, 26), (14, 3), (12, 24), (11, 1), (1, 5), (15, 4), (6, 1), (8, 20), (24, 13), (18, 6), (26, 19), (23, 11), (4, 16), (19, 7), (5, 17), (10, 1), (1, 7), (16, 5), (13, 24)}
Paths from source to sink:  {(1, 2, 14, 26), (1, 3, 15, 26), (1, 2, 14, 3, 15, 26)}
A best path in Gp is  (1, 3, 15, 26)  of minimum cost  17
The min capacity on this path is epsilon  6
The min cost is  17
(6, 18)  flow =  11
(21, 10)  flow =  0
(16, 26)  flow =  6
(22, 11)  flow =  0
(18, 26)  flow =  11
(20, 26)  flow =  16
(17, 6)  flow =  0
(22, 26)  flow = 

In [ ]:
#Twelfth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (12, 1), (22, 11), (5, 1), (1, 9), (11, 23), (7, 1), (21, 9), (26, 23), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (1, 2), (1, 11), (1, 8), (26, 16), (20, 9), (26, 25), (13, 25), (26, 22), (4, 1), (8, 1), (14, 26), (1, 4), (9, 21), (1, 13), (24, 12), (1, 10), (10, 22), (26, 18), (26, 15), (21, 10), (26, 24), (20, 8), (3, 1), (17, 6), (1, 6), (1, 3), (16, 4), (2, 14), (1, 12), (25, 13), (18, 7), (26, 20), (26, 17), (14, 3), (12, 24), (11, 1), (1, 5), (15, 4), (6, 1), (8, 20), (24, 13), (18, 6), (26, 19), (23, 11), (4, 16), (19, 7), (5, 17), (10, 1), (1, 7), (16, 5), (15, 3), (13, 24)}
Paths from source to sink:  {(1, 2, 14, 26)}
A best path in Gp is  (1, 2, 14, 26)  of minimum cost  18
The min capacity on this path is epsilon  9
The min cost is  18
(6, 18)  flow =  11
(21, 10)  flow =  0
(16, 26)  flow =  6
(22, 11)  flow =  0
(18, 26)  flow =  11
(20, 26)  flow =  16
(17, 6)  flow =  0
(22, 26)  flow =  15
(1, 6)  flow =  11

In [ ]:
#Thirteenth Iteration
Iterate(G3)

Incremental Network: {(26, 21), (6, 18), (12, 1), (22, 11), (5, 1), (1, 9), (11, 23), (7, 1), (26, 14), (21, 9), (26, 23), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (1, 2), (2, 1), (1, 11), (1, 8), (26, 16), (20, 9), (26, 25), (13, 25), (26, 22), (14, 2), (4, 1), (8, 1), (1, 4), (9, 21), (1, 13), (24, 12), (1, 10), (10, 22), (26, 18), (26, 15), (21, 10), (26, 24), (20, 8), (3, 1), (17, 6), (1, 6), (1, 3), (16, 4), (2, 14), (1, 12), (25, 13), (18, 7), (26, 20), (26, 17), (14, 3), (12, 24), (11, 1), (1, 5), (15, 4), (6, 1), (8, 20), (24, 13), (18, 6), (26, 19), (23, 11), (4, 16), (19, 7), (5, 17), (10, 1), (1, 7), (16, 5), (15, 3), (13, 24)}
Paths from source to sink:  set()
Maximal flow found: 125  with minimal cost  1384
(6, 18)  flow =  11
(21, 10)  flow =  0
(16, 26)  flow =  6
(22, 11)  flow =  0
(18, 26)  flow =  11
(20, 26)  flow =  16
(17, 6)  flow =  0
(22, 26)  flow =  15
(1, 6)  flow =  11
(1, 3)  flow =  6
(1, 9)  flow =  18
(2, 14)  flow =  9
(1, 12)  f

#Part B

In [ ]:
#Define the network
G4 ={(1,2),(2,3),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(1,11),(1,12),(1,13),(1,14),(3,15),(4,16),(5,17),(6,18),(7,19),(8,20),(9,21),(10,22),(11,23),(12,24),(13,25),(14,26),(15,4),(16,5),(17,6),(18,7),(19,8),(20,9),(21,10),(22,11),(23,12),(24,13),(25,14),(15,27),(16,27),(17,27),(18,27),(19,27),(20,27),(21,27),(22,27),(23,27),(24,27),(25,27),(26,27)}

In [ ]:
#Initialise the network and set all values to 0
Initialise(G4)

All values of c,phi and l initialised to zero


In [ ]:
#defining capacities
#from source to firm
c[1][2]=5
c[2][3]=5
# c[26][3]=5
c[1][3]=20
c[1][4]=20
c[1][5]=20
c[1][6]=20
c[1][7]=20
c[1][8]=20
c[1][9]=20
c[1][10]=20
c[1][11]=20
c[1][12]=20
c[1][13]=20
c[1][14]=20
c[3][15]=20
c[4][16]=20
c[5][17]=20
c[6][18]=20
c[7][19]=20
c[8][20]=20
c[9][21]=20
c[10][22]=20
c[11][23]=20
c[12][24]=20
c[13][25]=20
c[14][26]=20

#leftover
c[15][4]=20
c[16][5]=20
c[17][6]=20
c[18][7]=20
c[19][8]=20
c[20][9]=20
c[21][10]=20
c[22][11]=20
c[23][12]=20
c[24][13]=20
c[25][14]=20

#storage to consumption
c[15][27]=9
c[16][27]=6
c[17][27]=6
c[18][27]=7
c[19][27]=11
c[20][27]=14
c[21][27]=16
c[22][27]=18
c[23][27]=15
c[24][27]=10
c[25][27]=7
c[26][27]=11

In [ ]:
#defining cost
#from source to firm
l[1][2]=0
l[2][3]=0
l[1][3]=18
l[1][4]=17
l[1][5]=17
l[1][6]=15
l[1][7]=12
l[1][8]=8
l[1][9]=7
l[1][10]=6
l[1][11]=9
l[1][12]=12
l[1][13]=14
l[1][14]=17
l[3][15]=0
l[4][16]=0
l[5][17]=0
l[6][18]=0
l[7][19]=0
l[8][20]=0
l[9][21]=0
l[10][22]=0
l[11][23]=0
l[12][24]=0
l[13][25]=0
l[14][26]=0

#leftover
l[15][4]=1
l[16][5]=1
l[17][6]=2
l[18][7]=2
l[19][8]=3
l[20][9]=5
l[21][10]=6
l[22][11]=6
l[23][12]=5
l[24][13]=3
l[25][14]=2
# l[26][3]=0

#storage to consumption
l[14][26]=0
l[15][26]=0
l[16][26]=0
l[17][26]=0
l[18][26]=0
l[19][26]=0
l[20][26]=0
l[21][26]=0
l[22][26]=0
l[23][26]=0
l[24][26]=0
l[25][26]=0

In [ ]:
#First Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (22, 11), (26, 27), (15, 27), (17, 6), (1, 6), (1, 3), (1, 9), (17, 27), (11, 23), (1, 12), (19, 27), (18, 7), (7, 19), (23, 12), (21, 27), (12, 24), (3, 15), (23, 27), (19, 8), (1, 2), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (16, 27), (18, 27), (4, 16), (22, 27), (20, 27), (5, 17), (14, 26), (1, 4), (2, 3), (9, 21), (1, 7), (1, 13), (16, 5), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 9, 21, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 27), (1, 11, 23, 12, 24, 13, 25, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 27), (1, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 9, 21, 10, 22, 11, 23, 27), (1, 7, 19, 27), (1, 11, 23, 27), (1, 4, 16,

In [ ]:
#Second Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (22, 11), (26, 27), (15, 27), (17, 6), (1, 6), (1, 3), (1, 9), (17, 27), (11, 23), (1, 12), (19, 27), (18, 7), (7, 19), (23, 12), (21, 27), (12, 24), (3, 15), (23, 27), (19, 8), (27, 15), (2, 1), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (3, 2), (16, 27), (18, 27), (4, 16), (22, 27), (20, 27), (5, 17), (14, 26), (1, 4), (9, 21), (1, 7), (1, 13), (16, 5), (15, 3), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 9, 21, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 27), (1, 11, 23, 12, 24, 13, 25, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 27), (1, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 9, 21, 10, 22, 11, 23, 27), (1, 7, 19, 27), (1, 11, 

In [ ]:
#Third Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (22, 11), (26, 27), (15, 27), (17, 6), (1, 6), (1, 3), (1, 9), (27, 22), (17, 27), (11, 23), (1, 12), (19, 27), (18, 7), (7, 19), (23, 12), (22, 10), (21, 27), (12, 24), (3, 15), (23, 27), (19, 8), (27, 15), (2, 1), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (3, 2), (16, 27), (18, 27), (4, 16), (20, 27), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (15, 3), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 9, 21, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 27), (1, 11, 23, 12, 24, 13, 25, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 9, 21, 10, 22, 11, 23, 27), (1, 7, 19, 27), (1, 11, 23, 27), (1, 4, 16, 5, 17, 6, 18, 27), (

In [ ]:
#Fourth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (22, 11), (26, 27), (15, 27), (17, 6), (1, 6), (1, 3), (1, 9), (27, 22), (17, 27), (11, 23), (1, 12), (19, 27), (18, 7), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (23, 27), (19, 8), (27, 15), (27, 21), (2, 1), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (3, 2), (16, 27), (18, 27), (4, 16), (20, 27), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (15, 3), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 8, 20, 9, 21, 10

In [ ]:
#Fifth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (20, 8), (22, 11), (26, 27), (15, 27), (17, 6), (1, 6), (1, 3), (1, 9), (27, 22), (17, 27), (11, 23), (1, 12), (19, 27), (18, 7), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (23, 27), (19, 8), (27, 15), (27, 21), (2, 1), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (3, 2), (16, 27), (18, 27), (4, 16), (8, 1), (27, 20), (5, 17), (14, 26), (1, 4), (10, 1), (9, 21), (1, 7), (1, 13), (16, 5), (15, 3), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1,

In [ ]:
#Sixth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (20, 8), (22, 11), (26, 27), (15, 27), (17, 6), (1, 6), (1, 3), (1, 9), (27, 22), (17, 27), (11, 23), (1, 12), (19, 27), (18, 7), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (11, 1), (19, 8), (27, 15), (27, 21), (2, 1), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (23, 11), (3, 2), (16, 27), (18, 27), (4, 16), (8, 1), (27, 20), (5, 17), (14, 26), (27, 23), (1, 4), (9, 21), (10, 1), (1, 7), (1, 13), (16, 5), (15, 3), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 2

In [ ]:
#Seventh Iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (24, 27), (20, 8), (22, 11), (26, 27), (15, 27), (17, 6), (27, 19), (1, 6), (1, 3), (1, 9), (27, 22), (17, 27), (11, 23), (1, 12), (7, 1), (18, 7), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (11, 1), (19, 8), (27, 15), (27, 21), (2, 1), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (23, 11), (3, 2), (16, 27), (18, 27), (4, 16), (8, 1), (19, 7), (27, 20), (5, 17), (14, 26), (27, 23), (1, 4), (9, 21), (10, 1), (1, 7), (1, 13), (16, 5), (15, 3), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 11, 23, 12, 24, 27), (1, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 27), (1, 5, 17, 27), (1, 7, 19, 8, 20, 9, 21, 10, 

In [ ]:
#Eight iteration
Iterate(G4)

Incremental Network: {(6, 18), (21, 10), (12, 1), (20, 8), (22, 11), (26, 27), (15, 27), (17, 6), (27, 19), (1, 6), (1, 3), (1, 9), (27, 22), (17, 27), (11, 23), (1, 12), (7, 1), (18, 7), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (12, 24), (3, 15), (11, 1), (19, 8), (27, 15), (27, 21), (2, 1), (27, 24), (1, 5), (1, 11), (15, 4), (8, 20), (1, 8), (1, 14), (24, 13), (20, 9), (25, 27), (13, 25), (23, 11), (3, 2), (16, 27), (18, 27), (4, 16), (8, 1), (19, 7), (27, 20), (5, 17), (14, 26), (27, 23), (1, 4), (9, 21), (10, 1), (1, 7), (1, 13), (16, 5), (15, 3), (24, 12), (1, 10), (25, 14), (10, 22)}
Paths from source to sink:  {(1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 5, 17, 27), (1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 27), (1, 3, 15, 4, 16, 27), (1, 11, 23, 12, 24, 13, 25, 27), (1, 13, 25, 27), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 

In [ ]:
#Ninth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (26, 27), (15, 27), (27, 19), (1, 9), (17, 27), (11, 23), (7, 1), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (18, 27), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (17, 6), (1, 6), (27, 25), (1, 3), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (11, 1), (27, 15), (27, 24), (1, 5), (15, 4), (8, 20), (1, 14), (24, 13), (23, 11), (16, 27), (4, 16), (19, 7), (27, 20), (5, 17), (10, 1), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  {(1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 5, 17, 27), (1, 4, 16, 5, 17, 27), (1, 3, 15, 4, 16, 27), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25,

In [ ]:
#Tenth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (26, 27), (15, 27), (27, 19), (1, 9), (17, 27), (11, 23), (7, 1), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (17, 6), (1, 6), (27, 25), (1, 3), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (27, 18), (11, 1), (27, 15), (27, 24), (1, 5), (6, 1), (15, 4), (8, 20), (1, 14), (24, 13), (18, 6), (23, 11), (16, 27), (4, 16), (19, 7), (27, 20), (5, 17), (10, 1), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  {(1, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 5, 17, 27), (1, 4, 16, 5, 17, 27), (1, 3, 15, 4, 16, 27), (1, 4, 16, 5, 17, 6, 18, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 7, 19, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 14, 26, 27), (1, 8, 20, 9, 21, 10, 22, 11, 23

In [ ]:
#Eleventh Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (15, 27), (14, 25), (27, 19), (1, 9), (17, 27), (11, 23), (7, 1), (26, 14), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (17, 6), (1, 6), (27, 25), (1, 3), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (27, 18), (11, 1), (27, 15), (27, 24), (1, 5), (6, 1), (15, 4), (8, 20), (1, 14), (24, 13), (18, 6), (23, 11), (16, 27), (4, 16), (19, 7), (27, 20), (5, 17), (10, 1), (27, 26), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  {(1, 4, 16, 27), (1, 5, 17, 27), (1, 4, 16, 5, 17, 27), (1, 3, 15, 27), (1, 3, 15, 4, 16, 27), (1, 3, 15, 4, 16, 5, 17, 27)}
A best path in Gp is  (1, 4, 16, 27)  of minimum cost  17
The min capacity on this path is epsilon  6
The min cost is  17
(6, 18)  flow =  7
(21, 10)  flow =  0
(24, 27)  flow =  10
(22, 1

In [ ]:
#Twelfth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (15, 27), (14, 25), (27, 19), (1, 9), (17, 27), (11, 23), (7, 1), (26, 14), (21, 9), (7, 19), (23, 12), (22, 10), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (4, 1), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (17, 6), (27, 16), (1, 6), (27, 25), (1, 3), (16, 4), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (27, 18), (11, 1), (27, 15), (27, 24), (1, 5), (6, 1), (15, 4), (8, 20), (1, 14), (24, 13), (18, 6), (23, 11), (4, 16), (19, 7), (27, 20), (5, 17), (10, 1), (27, 26), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  {(1, 5, 17, 27), (1, 4, 16, 5, 17, 27), (1, 3, 15, 27), (1, 3, 15, 4, 16, 5, 17, 27)}
A best path in Gp is  (1, 5, 17, 27)  of minimum cost  17
The min capacity on this path is epsilon  6
The min cost is  17
(6, 18)  flow =  7
(21, 10)  flow =  0
(24, 27)  flow =  10
(22, 11)  flow =  0
(15, 27)

In [ ]:
#Thirteenth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (15, 27), (5, 1), (14, 25), (27, 19), (1, 9), (11, 23), (7, 1), (26, 14), (21, 9), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (4, 1), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (17, 6), (27, 16), (1, 6), (27, 25), (1, 3), (16, 4), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (27, 18), (11, 1), (27, 15), (27, 24), (1, 5), (6, 1), (15, 4), (8, 20), (1, 14), (24, 13), (18, 6), (23, 11), (4, 16), (19, 7), (27, 20), (5, 17), (27, 17), (10, 1), (27, 26), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  {(1, 3, 15, 27)}
A best path in Gp is  (1, 3, 15, 27)  of minimum cost  18
The min capacity on this path is epsilon  4
The min cost is  18
(6, 18)  flow =  7
(21, 10)  flow =  0
(24, 27)  flow =  10
(22, 11)  flow =  0
(15, 27)  flow =  9
(26, 27)  flow =  11
(17, 6)  flow =  0


In [ ]:
#Fourteenth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (5, 1), (14, 25), (27, 19), (1, 9), (11, 23), (7, 1), (26, 14), (21, 9), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (4, 1), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (3, 1), (17, 6), (27, 16), (1, 6), (27, 25), (1, 3), (16, 4), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (27, 18), (11, 1), (27, 15), (27, 24), (1, 5), (6, 1), (15, 4), (8, 20), (1, 14), (24, 13), (18, 6), (23, 11), (4, 16), (19, 7), (27, 20), (5, 17), (27, 17), (10, 1), (27, 26), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  set()
Maximal flow found: 130  with minimal cost  1374
(6, 18)  flow =  7
(21, 10)  flow =  0
(24, 27)  flow =  10
(22, 11)  flow =  0
(15, 27)  flow =  9
(26, 27)  flow =  11
(17, 6)  flow =  0
(1, 6)  flow =  7
(1, 3)  flow =  4
(1, 9)  flow =  16
(17, 27)  flow =  6
(11, 23)  f

In [ ]:
#Fifteenth Iteration
Iterate(G4)

Incremental Network: {(6, 18), (12, 1), (22, 11), (5, 1), (14, 25), (27, 19), (1, 9), (11, 23), (7, 1), (26, 14), (21, 9), (7, 19), (23, 12), (22, 10), (17, 5), (9, 1), (3, 15), (19, 8), (27, 21), (2, 1), (13, 1), (1, 11), (1, 8), (20, 9), (13, 25), (3, 2), (4, 1), (8, 1), (14, 26), (27, 23), (1, 4), (9, 21), (1, 13), (25, 14), (24, 12), (1, 10), (10, 22), (21, 10), (20, 8), (3, 1), (17, 6), (27, 16), (1, 6), (27, 25), (1, 3), (16, 4), (27, 22), (1, 12), (25, 13), (18, 7), (12, 24), (27, 18), (11, 1), (27, 15), (27, 24), (1, 5), (6, 1), (15, 4), (8, 20), (1, 14), (24, 13), (18, 6), (23, 11), (4, 16), (19, 7), (27, 20), (5, 17), (27, 17), (10, 1), (27, 26), (1, 7), (16, 5), (15, 3)}
Paths from source to sink:  set()
Maximal flow found: 130  with minimal cost  1374
(6, 18)  flow =  7
(21, 10)  flow =  0
(24, 27)  flow =  10
(22, 11)  flow =  0
(15, 27)  flow =  9
(26, 27)  flow =  11
(17, 6)  flow =  0
(1, 6)  flow =  7
(1, 3)  flow =  4
(1, 9)  flow =  16
(17, 27)  flow =  6
(11, 23)  f